# Py City Schools

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## Local Government Area Summary

* Criteria to be displayed include:

    * Total number of schools
    * Total number of students
    * Total budget
    * Average maths score 
    * Average reading score
    * Percentage of students with a passing maths score (50 or greater)
    * Percentage of students with a passing reading score (50 or greater)
    * Percentage of students who passed maths **and** reading (% Overall Passing)
    * Display dataframe to hold the above results

In [2]:
#----------Collect the summary data-------------

# Count the number of unique school names in school_data_complete dataframe
total_no_schools = len(school_data_complete["school_name"].unique())

# Count the number of unique student numbers in school_data_complete dataframe
total_no_students = len(school_data_complete["Student ID"].unique())

# Sum all data in the budget column in the school_data dataframe
total_budget = school_data["budget"].sum()

# Average the maths score column in school_data_complete dataframe
average_maths_score = school_data_complete["maths_score"].mean()

# Average the reading score column in school_data_complete dataframe
average_reading_score = school_data_complete["reading_score"].mean()

# Calculation on the percentage of students that got 50% or over in maths
perc_passing_maths = len(school_data_complete[school_data_complete["maths_score"] >= 50]) / total_no_students * 100

# Calculation on the percentage of students that got 50% or over in reading
perc_passing_reading = len(school_data_complete[school_data_complete["reading_score"] >= 50]) / total_no_students * 100

# Calculation on the percentage of students that got 50% or over in maths & reading
perc_passing_maths_and_reading = len(school_data_complete.loc[(school_data_complete["maths_score"] >= 50) &
                                    (school_data_complete["reading_score"] >= 50), :]) / total_no_students *100


#----------Collate the summary data-------------

# create new dataframe from summary data using a dictionary
local_government_summary_df = pd.DataFrame({"Total Schools": [total_no_schools],	
                                "Total Students": [total_no_students],
                            	"Total Budget": [total_budget],	
                                "Average Maths Score": [average_maths_score],
                            	"Average Reading Score": [average_reading_score],
                            	"% Passing Maths": [perc_passing_maths],
                            	"% Passing Reading": [perc_passing_reading],
                                "% Overall Passing": [perc_passing_maths_and_reading]})

#----------Format the summary data-------------

### format rows to dispay commas, $ or % where relative

# comma
local_government_summary_df["Total Students"] = local_government_summary_df["Total Students"].map("{:,}".format)

# comma, $ and 2 decimal places
local_government_summary_df["Total Budget"] = local_government_summary_df["Total Budget"].map("${:,.2f}".format)

# 2 decimal places
local_government_summary_df["Average Maths Score"] = local_government_summary_df["Average Maths Score"].map("{:.2f}".format)
local_government_summary_df["Average Reading Score"] = local_government_summary_df["Average Reading Score"].map("{:.2f}".format)

# 2 decimal places and %
local_government_summary_df["% Passing Maths"] = local_government_summary_df["% Passing Maths"].map("{:.2f}%".format)
local_government_summary_df["% Passing Reading"] = local_government_summary_df["% Passing Reading"].map("{:.2f}%".format)
local_government_summary_df["% Overall Passing"] = local_government_summary_df["% Overall Passing"].map("{:.2f}%".format)

#----------Display the summary data-------------

#Display new dataframe
local_government_summary_df

Total Schools Total Students    Total Budget Average Maths Score  \
0             15         39,170  $24,649,428.00               70.34   

  Average Reading Score % Passing Maths % Passing Reading % Overall Passing  
0                 69.98          86.08%            84.43%            72.81%

## School Summary

* Display key metrics about each school, including:

  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed maths **and** reading.)

In [3]:

#----------Starting dataframe for the summary data-------------

# Creating new dataframe from original dataframe to keep the summary information
school_summary_df = school_data


#----------Calculations for the per student budget-------------

# Calculate budget per student (budget $ divide by the total of students)
per_student_budget = school_data["budget"] / school_data["size"]

# Adding per student budget calculation to the summary dataframe
school_summary_df["Per Sudent Budget"] = per_student_budget


#----------Calculations for the maths and reading score averages-------------

# Goup the complete dataframe by school name
school_data_complete_grouped = school_data_complete.groupby("school_name")

#Grab the average scores for reading and maths from the grouped dataframe
maths_reading_score_mean = school_data_complete_grouped[["maths_score", "reading_score"]].mean() 

# Merge school summary dataframe and the average reading/writing scores dataframe
school_summary_df = pd.merge(maths_reading_score_mean, school_summary_df, how="left", on="school_name")


#----------Calculations for score over 50 for maths, reading and both maths and reading as a percentage-------------

# Copy original school_data_complete dataframe into new dataframe
pass_score = school_data_complete.copy()

# Create new columns and pass true or false into the column if score >= 50
pass_score.loc[pass_score["maths_score"] >= 50, "Pass Maths"] = True
pass_score.loc[pass_score["reading_score"] >= 50, "Pass reading"] = True
pass_score.loc[(pass_score["maths_score"] >= 50) & (pass_score["reading_score"] >= 50), "Pass Maths and Reading"] = True

# Group the dataframe by the school name
pass_maths_score_group = pass_score.groupby("school_name")

# Create new dataframe displaying the count of "TRUE" values ie. showing how many are >= 50 
pass_totals_df = pass_maths_score_group[["Student ID", "Pass Maths", "Pass reading", "Pass Maths and Reading"]].count()

# Using the new groupd dataframe calculate the % by dividing the counts by the total number of students
pass_totals_df["% Passing Maths"] = pass_totals_df["Pass Maths"] / pass_totals_df["Student ID"] * 100
pass_totals_df["% Passing Reading"] = pass_totals_df["Pass reading"] / pass_totals_df["Student ID"] * 100
pass_totals_df["% Overall Passing"] = pass_totals_df["Pass Maths and Reading"] / pass_totals_df["Student ID"] * 100

# Update the dataframe by only displaying the required columns
pass_totals_df = pass_totals_df[["% Passing Maths", "% Passing Reading", "% Overall Passing"]]

# Merge the updated dataframe with the summary dataframe
school_summary_df = pd.merge(school_summary_df, pass_totals_df, how="left", on="school_name")

#----------Format the summary data-------------

# Renaming columns
school_summary_df = school_summary_df.rename(columns={"type": "School Type", 
                                                        "size": "Total Students", 
                                                        "budget": "Total School Budget", 
                                                        "maths_score": "Average Maths Score", 
                                                        "reading_score": "Average Reading Score"})

# Setting the order of the columns
school_summary_df = school_summary_df[["school_name", "School Type", 
                                        "Total Students", "Total School Budget", 
                                        "Per Sudent Budget", "Average Maths Score", 
                                        "Average Reading Score", "% Passing Maths", 
                                        "% Passing Reading", "% Overall Passing"]]

# Setting the index to the school name and then dropping the index column name
school_summary_df.set_index(["school_name"], drop=True, append=False, inplace=True, verify_integrity=True)
school_summary_df.index.name = None

# Copy school summary dataframe to new dataframe for formatting
# This will alllows us to be able to use the summay dataframe later on
formatted_school_summary = school_summary_df.copy()

# comma
formatted_school_summary["Total Students"] = formatted_school_summary["Total Students"].map("{:,}".format)

# comma, $ and 2 decimal places
formatted_school_summary["Total School Budget"] = formatted_school_summary["Total School Budget"].map("${:,.2f}".format)
formatted_school_summary["Per Sudent Budget"] = formatted_school_summary["Per Sudent Budget"].map("${:,.2f}".format)

# 2 decimal places
formatted_school_summary["Average Maths Score"] = formatted_school_summary["Average Maths Score"].map("{:,.2f}".format)
formatted_school_summary["Average Reading Score"] = formatted_school_summary["Average Reading Score"].map("{:,.2f}".format)

# 2 decimal places and %
formatted_school_summary["% Passing Maths"] = formatted_school_summary["% Passing Maths"].map("{:,.2f}%".format)
formatted_school_summary["% Passing Reading"] = formatted_school_summary["% Passing Reading"].map("{:,.2f}%".format)
formatted_school_summary["% Overall Passing"] = formatted_school_summary["% Overall Passing"].map("{:,.2f}%".format)

#----------Display the summary data-------------

# Displaying the summary data
formatted_school_summary



School Type Total Students Total School Budget  \
Bailey High School      Government          4,976       $3,124,928.00   
Cabrera High School    Independent          1,858       $1,081,356.00   
Figueroa High School    Government          2,949       $1,884,411.00   
Ford High School        Government          2,739       $1,763,916.00   
Griffin High School    Independent          1,468         $917,500.00   
Hernandez High School   Government          4,635       $3,022,020.00   
Holden High School     Independent            427         $248,087.00   
Huang High School       Government          2,917       $1,910,635.00   
Johnson High School     Government          4,761       $3,094,650.00   
Pena High School       Independent            962         $585,858.00   
Rodriguez High School   Government          3,999       $2,547,363.00   
Shelton High School    Independent          1,761       $1,056,600.00   
Thomas High School     Independent          1,635       $1,043,130.00   
Wilson High School     Independent          2,283       $1,319,574.00   
Wright High School     Independent          1,800       $1,049,400.00   

                      Per Sudent Budget Average Maths Score  \
Bailey High School              $628.00               72.35   
Cabrera High School             $582.00               71.66   
Figueroa High School            $639.00               68.70   
Ford High School                $644.00               69.09   
Griffin High School             $625.00               71.79   
Hernandez High School           $652.00               68.87   
Holden High School              $581.00               72.58   
Huang High School               $655.00               68.94   
Johnson High School             $650.00               68.84   
Pena High School                $609.00               72.09   
Rodriguez High School           $637.00               72.05   
Shelton High School             $600.00               72.03   
Thomas High School              $638.00               69.58   
Wilson High School              $578.00               69.17   
Wright High School              $583.00               72.05   

                      Average Reading Score % Passing Maths % Passing Reading  \
Bailey High School                    71.01          91.64%            87.38%   
Cabrera High School                   71.36          90.85%            89.07%   
Figueroa High School                  69.08          81.65%            82.81%   
Ford High School                      69.57          82.44%            82.22%   
Griffin High School                   71.25          91.21%            88.49%   
Hernandez High School                 69.19          80.95%            81.88%   
Holden High School                    71.66          89.93%            88.52%   
Huang High School                     68.91          81.69%            81.45%   
Johnson High School                   69.04          82.06%            81.98%   
Pena High School                      71.61          91.68%            86.59%   
Rodriguez High School                 70.94          90.80%            87.40%   
Shelton High School                   70.26          91.54%            86.71%   
Thomas High School                    69.77          83.85%            82.63%   
Wilson High School                    68.88          82.79%            81.30%   
Wright High School                    70.97          91.78%            86.67%   

                      % Overall Passing  
Bailey High School               80.08%  
Cabrera High School              80.79%  
Figueroa High School             67.65%  
Ford High School                 67.47%  
Griffin High School              81.34%  
Hernandez High School            66.36%  
Holden High School               78.92%  
Huang High School                66.71%  
Johnson High School              67.19%  
Pena High School                 79.21%  
Rodriguez High School            79.42%  
Shelton High School              78.88%  
Th

## Top Performing Schools (By % Overall Passing)

In [4]:
# Sort column ("% Overall Passing") in decending order
# Limit the results to the top 5
formatted_school_summary.sort_values("% Overall Passing", ascending = False).head(5)

School Type Total Students Total School Budget  \
Griffin High School    Independent          1,468         $917,500.00   
Cabrera High School    Independent          1,858       $1,081,356.00   
Bailey High School      Government          4,976       $3,124,928.00   
Wright High School     Independent          1,800       $1,049,400.00   
Rodriguez High School   Government          3,999       $2,547,363.00   

                      Per Sudent Budget Average Maths Score  \
Griffin High School             $625.00               71.79   
Cabrera High School             $582.00               71.66   
Bailey High School              $628.00               72.35   
Wright High School              $583.00               72.05   
Rodriguez High School           $637.00               72.05   

                      Average Reading Score % Passing Maths % Passing Reading  \
Griffin High School                   71.25          91.21%            88.49%   
Cabrera High School                   71.36          90.85%            89.07%   
Bailey High School                    71.01          91.64%            87.38%   
Wright High School                    70.97          91.78%            86.67%   
Rodriguez High School                 70.94          90.80%            87.40%   

                      % Overall Passing  
Griffin High School              81.34%  
Cabrera High School              80.79%  
Bailey High School               80.08%  
Wright High School               79.72%  
Rodriguez High School            79.42%

## Bottom Performing Schools (By % Overall Passing)

In [5]:
# Sort column ("% Overall Passing") in ascending order
# Limit the results to the top 5
formatted_school_summary.sort_values("% Overall Passing").head(5)

School Type Total Students Total School Budget  \
Hernandez High School   Government          4,635       $3,022,020.00   
Huang High School       Government          2,917       $1,910,635.00   
Johnson High School     Government          4,761       $3,094,650.00   
Wilson High School     Independent          2,283       $1,319,574.00   
Ford High School        Government          2,739       $1,763,916.00   

                      Per Sudent Budget Average Maths Score  \
Hernandez High School           $652.00               68.87   
Huang High School               $655.00               68.94   
Johnson High School             $650.00               68.84   
Wilson High School              $578.00               69.17   
Ford High School                $644.00               69.09   

                      Average Reading Score % Passing Maths % Passing Reading  \
Hernandez High School                 69.19          80.95%            81.88%   
Huang High School                     68.91          81.69%            81.45%   
Johnson High School                   69.04          82.06%            81.98%   
Wilson High School                    68.88          82.79%            81.30%   
Ford High School                      69.57          82.44%            82.22%   

                      % Overall Passing  
Hernandez High School            66.36%  
Huang High School                66.71%  
Johnson High School              67.19%  
Wilson High School               67.46%  
Ford High School                 67.47%

## Maths Scores by Year

* Display a table that lists the average maths score for students of each year level (9, 10, 11, 12) at each school.

In [6]:
#----------Collect the summary data-------------

# Create series for each year from year 9 to year 12
year_9 = school_data_complete.loc[school_data_complete["year"] == 9]
year_10 = school_data_complete.loc[school_data_complete["year"] == 10]
year_11 = school_data_complete.loc[school_data_complete["year"] == 11]
year_12 = school_data_complete.loc[school_data_complete["year"] == 12]

#----------Group the summary data-------------

# Group each series by the school name
year_9_group = year_9.groupby("school_name")
year_10_group = year_10.groupby("school_name")
year_11_group = year_11.groupby("school_name")
year_12_group = year_12.groupby("school_name")

#----------Get average of the summary data-------------

# Use .mean to calculate the maths average while still grouped by school name
year_9_math_average = year_9_group["maths_score"].mean()
year_10_math_average = year_10_group["maths_score"].mean()
year_11_math_average = year_11_group["maths_score"].mean()
year_12_math_average = year_12_group["maths_score"].mean()

#----------Join the summary data-------------

# Create a dataframe of the series created above using the .concat function
maths_score_by_year_df = pd.concat([year_9_math_average, 
                                    year_10_math_average, 
                                    year_11_math_average, 
                                    year_12_math_average], 
                                    axis=1)

#----------Format the summary data-------------

# Rename columns for easier readability
maths_score_by_year_df.columns.values[0] = "Year 9"
maths_score_by_year_df.columns.values[1] = "Year 10"
maths_score_by_year_df.columns.values[2] = "Year 11"
maths_score_by_year_df.columns.values[3] = "Year 12"

#dropping the index column name
maths_score_by_year_df.index.name = None

# Formating the visual data to 2 decimal places
maths_score_by_year_df["Year 9"] = maths_score_by_year_df["Year 9"].map("{:,.2f}".format)
maths_score_by_year_df["Year 10"] = maths_score_by_year_df["Year 10"].map("{:,.2f}".format)
maths_score_by_year_df["Year 11"] = maths_score_by_year_df["Year 11"].map("{:,.2f}".format)
maths_score_by_year_df["Year 12"] = maths_score_by_year_df["Year 12"].map("{:,.2f}".format)

#----------Display the summary data-------------

maths_score_by_year_df

Year 9 Year 10 Year 11 Year 12
Bailey High School     72.49   71.90   72.37   72.68
Cabrera High School    72.32   72.44   71.01   70.60
Figueroa High School   68.48   68.33   68.81   69.33
Ford High School       69.02   69.39   69.25   68.62
Griffin High School    72.79   71.09   71.69   71.47
Hernandez High School  68.59   68.87   69.15   68.99
Holden High School     70.54   75.11   71.64   73.41
Huang High School      69.08   68.53   69.43   68.64
Johnson High School    69.47   67.99   68.64   69.29
Pena High School       72.00   72.40   72.52   71.19
Rodriguez High School  71.94   71.78   72.36   72.15
Shelton High School    72.93   72.51   70.10   72.33
Thomas High School     69.23   70.06   69.66   69.37
Wilson High School     69.21   69.46   68.38   69.79
Wright High School     71.74   72.18   73.28   70.85

## Reading Score by Year

* Display a table that lists the average reading score for students of each year level (9, 10, 11, 12) at each school

In [7]:
#----------Using the grouped summary data calculation from previous-------------

# Use .mean to calculate the reading average while still grouped by school name
year_9_reading_average = year_9_group["reading_score"].mean()
year_10_reading_average = year_10_group["reading_score"].mean()
year_11_reading_average = year_11_group["reading_score"].mean()
year_12_reading_average = year_12_group["reading_score"].mean()

#----------Join the summary data-------------

# Create a dataframe of the series created above using the .concat function
reading_score_by_year_df = pd.concat([year_9_reading_average, 
                                    year_10_reading_average, 
                                    year_11_reading_average, 
                                    year_12_reading_average], 
                                    axis=1)

#----------Format the summary data-------------

# Rename columns for easier readability
reading_score_by_year_df.columns.values[0] = "Year 9"
reading_score_by_year_df.columns.values[1] = "Year 10"
reading_score_by_year_df.columns.values[2] = "Year 11"
reading_score_by_year_df.columns.values[3] = "Year 12"

#dropping the index column name
reading_score_by_year_df.index.name = None

# Formating the visual data to 2 decimal places
reading_score_by_year_df["Year 9"] = reading_score_by_year_df["Year 9"].map("{:,.2f}".format)
reading_score_by_year_df["Year 10"] = reading_score_by_year_df["Year 10"].map("{:,.2f}".format)
reading_score_by_year_df["Year 11"] = reading_score_by_year_df["Year 11"].map("{:,.2f}".format)
reading_score_by_year_df["Year 12"] = reading_score_by_year_df["Year 12"].map("{:,.2f}".format)

#----------Display the summary data-------------

reading_score_by_year_df

Year 9 Year 10 Year 11 Year 12
Bailey High School     70.90   70.85   70.32   72.20
Cabrera High School    71.17   71.33   71.20   71.86
Figueroa High School   70.26   67.68   69.15   69.08
Ford High School       69.62   68.99   70.74   68.85
Griffin High School    72.03   70.75   72.39   69.43
Hernandez High School  68.48   70.62   68.42   69.24
Holden High School     71.60   71.10   73.31   70.48
Huang High School      68.67   69.52   68.74   68.67
Johnson High School    68.72   69.30   69.97   67.99
Pena High School       70.95   72.32   71.70   71.51
Rodriguez High School  70.90   70.14   71.42   71.41
Shelton High School    70.72   69.88   71.15   69.07
Thomas High School     69.67   69.74   70.75   68.73
Wilson High School     68.68   68.41   68.80   69.89
Wright High School     71.82   71.39   70.48   69.82

## Scores by School Spending

* Display a table that breaks down school performances based on average Spending Ranges (Per Student). 

In [8]:
#----------Creation of the bins and labels-------------

# Create the ranges(bins) 
bins = [0, 584, 629, 644, 679]
# Create the labels
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

#----------Calculations-------------

# Add new column and use bins and labels to fill the column data based on the $ per student budget
school_summary_df["Spending Ranges (Per Student)"] = pd.cut(school_summary_df["Per Sudent Budget"], bins, labels=labels)

#----------Grouping-------------

# Goup the data by the new column "Spending Ranges (Per Student)"
scores_by_school_spending_group = school_summary_df.groupby("Spending Ranges (Per Student)").mean()

#----------Format the grouped data-------------

# 2 decimal places
scores_by_school_spending_group["Average Maths Score"] = scores_by_school_spending_group["Average Maths Score"].map("{:,.2f}".format)
scores_by_school_spending_group["Average Reading Score"] = scores_by_school_spending_group["Average Reading Score"].map("{:,.2f}".format)

# 2 decimal places and %
scores_by_school_spending_group["% Passing Maths"] = scores_by_school_spending_group["% Passing Maths"].map("{:,.2f}%".format)
scores_by_school_spending_group["% Passing Reading"] = scores_by_school_spending_group["% Passing Reading"].map("{:,.2f}%".format)
scores_by_school_spending_group["% Overall Passing"] = scores_by_school_spending_group["% Overall Passing"].map("{:,.2f}%".format)

#----------Display the grouped data-------------

scores_by_school_spending_group[["Average Maths Score", "Average Reading Score", "% Passing Maths", "% Passing Reading", "% Overall Passing"]]

Average Maths Score Average Reading Score  \
Spending Ranges (Per Student)                                             
<$585                                       71.36                 70.72   
$585-630                                    72.07                 71.03   
$630-645                                    69.85                 69.84   
$645-680                                    68.88                 69.05   

                              % Passing Maths % Passing Reading  \
Spending Ranges (Per Student)                                     
<$585                                  88.84%            86.39%   
$585-630                               91.52%            87.29%   
$630-645                               84.69%            83.76%   
$645-680                               81.57%            81.77%   

                              % Overall Passing  
Spending Ranges (Per Student)                    
<$585                                    76.72%  
$585-630                                 79.88%  
$630-645                                 71.00%  
$645-680                                 66.76%

## Scores by School Size

* Display a table that breaks down school performances based on school size.

In [9]:
#----------Creation of the bins and labels-------------

# Create the ranges(bins) 
bins = [0, 999, 1999, 4999]
# Create the labels
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

#----------Calculations-------------

# Add new column and use bins and labels to fill the column data based on the $ per student budget
school_summary_df["School Size"] = pd.cut(school_summary_df["Total Students"], bins, labels=labels)

#----------Grouping-------------

# Goup the data by the new column "School Size"
scores_by_school_size_group = school_summary_df.groupby("School Size").mean()

#----------Format the grouped data-------------

# 2 decimal places
scores_by_school_size_group["Average Maths Score"] = scores_by_school_size_group["Average Maths Score"].map("{:,.2f}".format)
scores_by_school_size_group["Average Reading Score"] = scores_by_school_size_group["Average Reading Score"].map("{:,.2f}".format)

# 2 decimal places and %
scores_by_school_size_group["% Passing Maths"] = scores_by_school_size_group["% Passing Maths"].map("{:,.2f}%".format)
scores_by_school_size_group["% Passing Reading"] = scores_by_school_size_group["% Passing Reading"].map("{:,.2f}%".format)
scores_by_school_size_group["% Overall Passing"] = scores_by_school_size_group["% Overall Passing"].map("{:,.2f}%".format)

#----------Display the grouped data-------------

scores_by_school_size_group[["Average Maths Score", "Average Reading Score", "% Passing Maths", "% Passing Reading", "% Overall Passing"]]

Average Maths Score Average Reading Score % Passing Maths  \
School Size                                                                    
Small (<1000)                    72.34                 71.64          90.81%   
Medium (1000-2000)               71.42                 70.72          89.85%   
Large (2000-5000)                69.75                 69.58          84.25%   

                   % Passing Reading % Overall Passing  
School Size                                             
Small (<1000)                 87.56%            79.07%  
Medium (1000-2000)            86.71%            78.04%  
Large (2000-5000)             83.30%            70.29%

## Scores by School Type

* Display a table that breaks down school performances based on school type

In [10]:

# Goup the data by the column "School ype"
scores_by_school_type_group = school_summary_df.groupby("School Type").mean()

#----------Format the grouped data-------------

# 2 decimal places
scores_by_school_type_group["Average Maths Score"] = scores_by_school_type_group["Average Maths Score"].map("{:,.2f}".format)
scores_by_school_type_group["Average Reading Score"] = scores_by_school_type_group["Average Reading Score"].map("{:,.2f}".format)

# 2 decimal places and %
scores_by_school_type_group["% Passing Maths"] = scores_by_school_type_group["% Passing Maths"].map("{:,.2f}%".format)
scores_by_school_type_group["% Passing Reading"] = scores_by_school_type_group["% Passing Reading"].map("{:,.2f}%".format)
scores_by_school_type_group["% Overall Passing"] = scores_by_school_type_group["% Overall Passing"].map("{:,.2f}%".format)

#----------Display the grouped data-------------

scores_by_school_type_group[["Average Maths Score", "Average Reading Score", "% Passing Maths", "% Passing Reading", "% Overall Passing"]]

Average Maths Score Average Reading Score % Passing Maths  \
School Type                                                             
Government                69.83                 69.68          84.46%   
Independent               71.37                 70.72          89.20%   

            % Passing Reading % Overall Passing  
School Type                                      
Government             83.59%            70.70%  
Independent            86.25%            76.97%

## Conclusions

* Independent school students have a higher pass rate than Government schools.
* The smaller the school, the more likely the student is to pass both maths and reading.
* It does not appear that budget positively impacts the student success rate.